# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lebu,-37.6167,-73.6500,9.70,88,100,3.21,CL,1690018494
1,1,blackmans bay,-43.0167,147.3167,10.85,82,100,2.67,AU,1690018494
2,2,adamstown,-25.0660,-130.1015,20.96,61,100,8.01,PN,1690018494
3,3,belmonte,-15.8631,-38.8828,21.33,87,82,4.79,BR,1690018495
4,4,luderitz,-26.6481,15.1594,17.23,41,100,2.21,NaN,1690018495


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    # alternate tiles, pretty, but distracting from the data points
    # tiles = "EsriImagery",
    # tiles = "EsriNatGeo",
    color="City",
    size="Humidity",
    frame_width=800,
    frame_height=600,
)

# Display the map
map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
df = city_data_df[(city_data_df["Max Temp"] > 25) & (city_data_df["Humidity"] > 50) & (city_data_df["Cloudiness"] < 30)]

# Drop any rows with null values
# this seems redundant - wasn't this done when we skipped cities early in WeatherPy?
# yeah, i keep checking - no NaN in "output_data/cities.csv"
df = df.dropna(how="any")

# Display sample data
df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,cabo san lucas,22.8909,-109.9124,30.34,63,4,1.34,MX,1690018496
29,29,constantia,44.1833,28.6500,32.61,53,15,7.93,RO,1690018496
35,35,fale old settlement,-9.3852,-171.2468,28.46,73,9,3.35,TK,1690018497
50,50,jayapura,-2.5333,140.7000,27.66,85,15,1.15,ID,1690018498
81,81,boda,4.3189,17.4695,30.26,61,18,1.00,CF,1690018500


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
18,cabo san lucas,MX,22.8909,-109.9124,63,
29,constantia,RO,44.1833,28.6500,53,
35,fale old settlement,TK,-9.3852,-171.2468,73,
50,jayapura,ID,-2.5333,140.7000,85,
81,boda,CF,4.3189,17.4695,61,
84,saint croix,VI,17.7397,-64.7388,78,
99,mermoz boabab,SN,14.7065,-17.4758,65,
100,road town,VG,18.4167,-64.6167,77,
133,sittwe,MM,20.1500,92.9000,78,
150,hamada,JP,34.8833,132.0833,86,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey" : geoapify_key,
    "categories" : "accommodation.hotel",
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cabo san lucas - nearest hotel: Comfort Rooms
constantia - nearest hotel: Hotel Chérica
fale old settlement - nearest hotel: No hotel found
jayapura - nearest hotel: Hotel Jasmine Jayapura
boda - nearest hotel: No hotel found
saint croix - nearest hotel: The Buccaneer
mermoz boabab - nearest hotel: Le Shéhérazade
road town - nearest hotel: No hotel found
sittwe - nearest hotel: Yuzana Aung Motel 1
hamada - nearest hotel: キャッスルホテル
longboat key - nearest hotel: Zota Beach Resort
gorontalo - nearest hotel: Grand City Hotel
atafu village - nearest hotel: No hotel found
george town - nearest hotel: Page 63 hostel
georgetown - nearest hotel: Page 63 hostel
kwang binh - nearest hotel: Nhà hàng Amor Palace
taulaga - nearest hotel: No hotel found
ingleside - nearest hotel: Quality Inn
dhromolaxia - nearest hotel: Helen of Troy gardens
tual - nearest hotel: Suita
makassar - nearest hotel: Pondok Dahlia House
frontera - nearest hotel: Quirino
kuala terengganu - nearest hotel

,City,Country,Lat,Lng,Humidity,Hotel Name
18,cabo san lucas,MX,22.8909,-109.9124,63,Comfort Rooms
29,constantia,RO,44.1833,28.6500,53,Hotel Chérica
35,fale old settlement,TK,-9.3852,-171.2468,73,No hotel found
50,jayapura,ID,-2.5333,140.7000,85,Hotel Jasmine Jayapura
81,boda,CF,4.3189,17.4695,61,No hotel found
84,saint croix,VI,17.7397,-64.7388,78,The Buccaneer
99,mermoz boabab,SN,14.7065,-17.4758,65,Le Shéhérazade
100,road town,VG,18.4167,-64.6167,77,No hotel found
133,sittwe,MM,20.1500,92.9000,78,Yuzana Aung Motel 1
150,hamada,JP,34.8833,132.0833,86,キャッスルホテル


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    color="City",
    line_color="black",
    size=80,
    frame_width=800,
    frame_height=600,
    hover_cols=["Hotel Name", "Country", "Humidity"]
)

# Display the map
map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country,Humidity)